In [13]:
import copy
import os
from contextlib import ExitStack
from typing import List

import pyro
import pytest
import torch
from chirho.counterfactual.handlers import MultiWorldCounterfactual
from chirho.interventional.handlers import do
from pyro.infer import Predictive
from torch.utils.data import DataLoader

from cities.modeling.svi_inference import run_svi_inference
from cities.modeling.zoning_models.zoning_tracts_model import TractsModel
from cities.modeling.zoning_models.zoning_tracts_sqm_model import TractsModelSqm
from cities.utils.data_grabber import find_repo_root
from cities.utils.data_loader import select_from_data

root = find_repo_root()

n_steps = 10
num_samples = 10


data_path = os.path.join(root, "data/minneapolis/processed/pg_census_tracts_dataset.pt")

dataset_read = torch.load(data_path)


loader = DataLoader(dataset_read, batch_size=len(dataset_read), shuffle=True)

data = next(iter(loader))


kwargs = {
    "categorical": ["year", "census_tract"],
    "continuous": {
        "housing_units",
        "total_value",
        "median_value",
        "mean_limit_original",
        "median_distance",
        "income",
        "segregation_original",
        "white_original",
        "parcel_mean_sqm",
        "parcel_median_sqm",
        "parcel_sqm",
    },
    "outcome": "housing_units",
}

pg_subset = select_from_data(data, kwargs)
pg_dataset_read = torch.load(data_path)

print("shape for pg", pg_subset["categorical"]["year"].shape)

def context_stack(contexts: List):

    stack = ExitStack()
    for context in contexts:
        stack.enter_context(context)
    return stack


def assert_no_repeated_non_ones(tensor_size):

    non_ones = [dim for dim in tensor_size if dim != 1]
    assert len(non_ones) == len(
        set(non_ones)
    ), f"Repeated non-1 dimensions found in {tensor_size}"

shape for pg torch.Size([1130])


/var/folders/pq/ntbz4ft13jv662kczkzk32pr0000gn/T/ipykernel_51333/3433613681.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset_read = torch.load(data_path)
/var/fo

In [24]:
from chirho.observational.handlers.predictive import PredictiveModel

# @pytest.mark.parametrize(
#     "model_class",
#     [TractsModel, TractsModelSqm],
# )
# def test_plated_sample_shaping(use_plate, use_mwc, use_do, model_class):
use_plate, use_mwc, use_do = True, True, True
model_class = TractsModelSqm

model = model_class(**data, categorical_levels=pg_dataset_read.categorical_levels)

# run forward to g
units = model(**data)
n = units.shape[0]
print(n)

pyro.clear_param_store()
guide = run_svi_inference(model, n_steps=n_steps, lr=0.03, plot=False, **data)

predictive = PredictiveModel(model, guide=guide)

data_for_preds = copy.deepcopy(data)
data_for_preds["continuous"]["housing_units"] = None
data_no_categorical_info = copy.deepcopy(data_for_preds)
data_no_categorical_info["categorical"]["year"] = None

context_managers = []

if use_plate:
    context_managers.append(pyro.plate("outer_plate", 4, dim=-8))
if use_mwc:
    context_managers.append(MultiWorldCounterfactual())
if use_do:
    context_managers.append(
        do(actions={"limit": (torch.tensor(0.0), torch.tensor(1.0))})
    )


with context_stack(context_managers):
    with pyro.poutine.trace() as tr_conditioned:
        samples_contextualized = predictive(**data_for_preds)
        samples_conditioned_contextualized = predictive(**data)
        samples_no_categorical_info = predictive(**data_no_categorical_info)

assert samples_conditioned_contextualized["housing_units"].shape == torch.Size(
    [num_samples, n]
)

assert_no_repeated_non_ones(samples_contextualized["housing_units"].shape)
assert_no_repeated_non_ones(samples_no_categorical_info["housing_units"].shape)

1130
[iteration 0001] loss: 300891.9062


RuntimeError: Multiple sample sites named '_AutoMultivariateNormal_latent'